In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=False,
)

device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_8bit=True
)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
model_id = "/home/lars/ML/LLM/results/checkpoint-30"
model = PeftModel.from_pretrained(model, model_id)
model.config.use_cache = False



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/lars/miniconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/lars/miniconda3/envs/llm/lib/libcudart.so.11.0'), PosixPath('/home/lars/miniconda3/envs/llm/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# %%timeit -r 5
# inference test
with torch.inference_mode():
    text = '### Human: Write a short dialogue between two Norwegian dudes, Lars and Victor, who are old friends, write it in Norwegian.### Assitant:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids, 
                            max_new_tokens=100, 
                            temperature=0.8, 
                            num_return_sequences=1,
                            do_sample=False,
                            use_cache = True,
                            top_k=50)

print(tokenizer.batch_decode(outputs))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['### Human: Write a short dialogue between two Norwegian dudes, Lars and Victor, who are old friends, write it in Norwegian.### Assitant:\nLars: Hei Victor, hvordan går det i dag da? Har du hatt en bra helg?\nVictor: Hei Lars! Ja, det går fint, takk. Helgen har vært fin, vi dro på en liten fjelltur med familien. Hva med deg?\nLars: Åh, det høres fantastisk ut! Jeg kom akkurat tilbake fra en liten fj']


In [66]:
## So inference time is probably significantly slower than the openai api, but interesting to experiment with further



NameError: name 'outputs' is not defined

In [14]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

Found cached dataset json (/home/lars/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-c93588435bc90172/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [53]:
import json
from datasets import load_dataset

# Read the JSON data
with open('data.json') as f:
    data = json.load(f)

# Transform the data into a list of dictionaries, each with a single key-value pair
transformed_data = [{"id": key, "text": value} for key, value in data.items()]

# Save the transformed data to a new JSON file
with open('transformed.json', 'w') as f:
    for item in transformed_data:
        json.dump(item, f)
        f.write('\n')

# Load the transformed data into a HuggingFace Dataset
dataset = load_dataset('json', data_files='transformed.json', split='train')

# Let's check the first element in the dataset
print(dataset[0])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/lars/.cache/huggingface/datasets/json/default-c043aa66b41a2909/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.
{'id': '0', 'text': '### Human Please generate a dialogue between Lisbeth, a young single mother, and Eirik, a restaurant owner discussing their week. Keep it colloquial. ### Assistant Lisbeth: Hei Eirik, hvordan har uka di vært?\nEirik: Hei Lisbeth! Uka har vært hektisk, altså. Vi har hatt mye å gjøre på restauranten. Og du da, hvordan har uka di vært med den lille?\nLisbeth: Åh, vet du, det har vært både oppturer og nedturer. Jeg elsker jo å være mamma, men det kan være litt slitsomt å gjøre alt på egenhånd, ass.\nEirik: Jeg kan tenke meg det, men du er jo en supermamma, så jeg er sikker på at du takler det bra.\nLisbeth: Åh, takk Eirik, det betyr mye å høre det! Så, hvordan går det med restauranten da? Har du noen nye retter på menyen?\nEirik: Ja, faktisk! Vi har lag

In [55]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [59]:
## Finetune

from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)



In [60]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [61]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/lars/miniconda3/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [62]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [63]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.800500
20,0.215300
30,0.017000


KeyboardInterrupt: 